In [1]:
import pandas as pd

import numpy as np

from sklearn.decomposition import PCA

from scipy.linalg import hankel, eigh

In [2]:
path = '../sensor/GetData/steps.txt'

In [3]:
df = pd.read_csv(path, skiprows=1, header=None,usecols=[1,2])

In [6]:
np.array(df).ravel()

array([ 0.000e+00, -0.000e+00,  2.100e-01,  4.700e-01,  3.900e-01,
        1.740e+00, -2.100e-01,  2.800e+00, -1.100e+00,  3.410e+00,
       -1.350e+00,  3.700e+00, -7.900e-01,  4.360e+00,  3.200e-01,
        4.810e+00,  1.550e+00,  4.650e+00,  2.700e+00,  4.210e+00,
        3.850e+00,  3.820e+00,  4.990e+00,  3.370e+00,  6.130e+00,
        2.960e+00,  7.290e+00,  2.600e+00,  8.480e+00,  2.210e+00,
        9.620e+00,  1.750e+00,  1.077e+01,  1.280e+00,  1.195e+01,
        9.200e-01,  1.309e+01,  4.300e-01,  1.427e+01,  1.300e-01,
        1.547e+01, -1.900e-01,  1.661e+01, -6.500e-01,  1.775e+01,
       -1.050e+00,  1.888e+01, -1.380e+00,  2.002e+01, -1.740e+00,
        2.115e+01, -2.110e+00,  2.227e+01, -2.490e+00,  2.341e+01,
       -2.810e+00,  2.442e+01, -3.120e+00,  2.449e+01, -2.790e+00,
        2.367e+01, -2.170e+00,  2.254e+01, -2.050e+00,  2.137e+01,
       -1.950e+00,  2.016e+01, -1.750e+00,  1.898e+01, -1.430e+00,
        1.779e+01, -1.100e+00,  1.661e+01, -7.500e-01,  1.539e

In [24]:
pca = PCA(n_components = 1)

In [25]:
df.head()

,1,2
0,0.00,-0.00
1,0.21,0.47
2,0.39,1.74
3,-0.21,2.80
4,-1.10,3.41


In [28]:
df = pca.fit_transform(df)

In [29]:
df

array([[-1.36990535],
       [-1.21515819],
       [-1.18190516],
       [-1.89944284],
       [-2.85349276],
       [-3.13508355],
       [-2.6544197 ],
       [-1.60331124],
       [-0.36307856],
       [ 0.82977346],
       [ 2.01689473],
       [ 3.2009588 ],
       [ 4.38043827],
       [ 5.57405521],
       [ 6.80091289],
       [ 7.9861231 ],
       [ 9.18241356],
       [10.3958987 ],
       [11.58454735],
       [12.7911556 ],
       [14.01992435],
       [15.20513456],
       [16.38346789],
       [17.54384408],
       [18.71759281],
       [19.8825536 ],
       [21.03872643],
       [22.20789057],
       [23.24676526],
       [23.27848109],
       [22.39282371],
       [21.25651662],
       [20.08276543],
       [18.85781636],
       [17.64891582],
       [16.42893503],
       [15.21659604],
       [13.96910524],
       [12.74683215],
       [11.5325821 ],
       [10.31412869],
       [ 9.05784994],
       [ 7.83901529],
       [ 6.5219858 ],
       [ 5.28786754],
       [ 4

In [33]:
len(df)

408

In [36]:
N = len(df)
L = N // 2
X_train = hankel(df[:L],df[L-1:]) # Creating trajectory matrix

In [39]:
eigenValues, eigenVectors = eigh(np.matmul(X_train, X_train.T))
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:,idx]

In [40]:
r = 1
# Extracted Training signals
U, Sigma, V = np.linalg.svd(X_train)
V = V.T
# d = np.linalg.matrix_rank(X_train)
X_elem = np.array( [Sigma[i] * np.outer(U[:,i], V[:,i]) for i in range(0,r)] )
X_train_extracted = X_elem.sum(axis=0)
X_train_extracted_data = np.asarray(list(X_train_extracted[:,0]) + list(X_train_extracted[:,-1]))

U = eigenVectors[:,:r] # r as statistical dimension
UT = U.T
pX = np.matmul(UT,X_train_extracted)
centroid = np.mean(pX, axis=1)
centroid = centroid[:,np.newaxis]

In [43]:
# Calculating the departure threshold in signal subspace using centroid and UT

#For training phase
Xtrg = hankel(df[:L], df[L-1:])
pXtrg = np.matmul(UT,Xtrg)
dtrg_matrix = centroid - pXtrg
dtrg_scores = np.linalg.norm(dtrg_matrix, axis=0, ord=2)

# For Validation phase and threshold calculation
Xt = hankel(df[:L],df[L-1:])
pXt = np.matmul(UT,Xt)
dt_matrix = centroid - pXt
dt_scores = np.linalg.norm(dt_matrix, axis=0, ord=2)
# d_scores = np.asarray([np.matmul(d_matrix[:,i].T, d_matrix[:,i]) for i in range(d_matrix.shape[1])])
dt_theta = np.max(dt_scores)
dt_theta

81.91067549313199

In [44]:
Xj = hankel(df[:L],df[L-1:])
pXj = np.matmul(UT, Xj)
dj_matrix = centroid - pXj
dj_scores = np.linalg.norm(dj_matrix, axis=0, ord=2)
# dj_scores = [np.matmul(dj_matrix[:,i].T, dj_matrix[:,i]) for i in range(dj_matrix.shape[1])]
dj_scores = np.asarray(dj_scores)
np.max(dj_scores)

81.91067549313199